In [18]:
from google.colab import files
uploaded= files.upload()

Saving US_youtube_sample.csv to US_youtube_sample (1).csv


In [67]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [73]:
file_path = 'US_youtube_sample.csv'
df = pd.read_csv(file_path)
df.reset_index(inplace=True, drop=True)
df.head()

,category_id,views,comments_disabled,ratings_disabled,channel_text,difference_d,difference_h,published_year,published_month,trending_year,trending_month
0,Entertainment,973398,"""False""","""False""",Are Jessica Chastain & Bryce Dallas Howard The...,11.0,272.0,2018,5,2018,6
1,Entertainment,706352,"""False""","""False""",ASKING MY CRUSH TO FLY TO PARIS FOR DREAM DATE...,5.0,126.0,2018,3,2018,3
2,Gaming,505198,"""False""","""False""",SURPRISING HER WITH A NEW PUPPY (8 WEEKS OLD) ...,5.0,125.0,2018,3,2018,3
3,News and Politics,283745,"""False""","""False""",The guide book that helped black Americans tra...,2.0,59.0,2018,3,2018,3
4,Entertainment,272604,"""False""","""False""",SPAGHETTI BURRITO VS SPAGHETTI BURRITO Hellthy...,2.0,57.0,2017,11,2017,11


In [74]:
#convert views to integer
df['views']=df['views'].astype(int)

In [75]:
#binning needed


#replace in dataframe
for v in df['views']:
  if v <= 10000:
    df.views = df.views.replace(v,str('< 10,000'))
  elif v < 100000:
    df.views = df.views.replace(v,str('10,001-99,999'))
  elif v < 500000:
    df.views = df.views.replace(v,str('100,000-499,999'))
  elif v < 1000000:
    df.views = df.views.replace(v, str('500,000-999,999'))
  else:
    df.views = df.views.replace(v,str('1,000,000+'))

In [76]:
#create the features
X = pd.get_dummies(df.drop(columns='views'))

#create the target
y= df['views']

In [77]:
y.value_counts()

1,000,000+         482
100,000-499,999    367
500,000-999,999    234
10,001-99,999      117
Name: views, dtype: int64

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(900, 1115)

In [79]:
print(Counter(y_train))
print(Counter(y_test))

Counter({'1,000,000+': 362, '100,000-499,999': 275, '500,000-999,999': 175, '10,001-99,999': 88})
Counter({'1,000,000+': 120, '100,000-499,999': 92, '500,000-999,999': 59, '10,001-99,999': 29})


In [80]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [81]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.44601184577202924

In [82]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[58, 22, 14, 26],
       [ 1, 19,  4,  5],
       [14, 25, 36, 17],
       [18, 17,  9, 15]])

In [83]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

     1,000,000+       0.64      0.48      0.82      0.55      0.63      0.38       120
  10,001-99,999       0.23      0.66      0.76      0.34      0.71      0.50        29
100,000-499,999       0.57      0.39      0.87      0.46      0.58      0.32        92
500,000-999,999       0.24      0.25      0.80      0.25      0.45      0.19        59

    avg / total       0.50      0.43      0.82      0.44      0.59      0.34       300



In [84]:
# List the features sorted in descending order by feature importance
features_rank = sorted(zip(brfc.feature_importances_, X.columns), reverse=True)
for feature in features_rank:
    print(f'{feature[1]}: ({feature[0]})')

difference_h: (0.11311215970870343)
difference_d: (0.08435175820260517)
published_month: (0.06078080918700026)
trending_month: (0.06070477484429357)
category_id_Music: (0.018520837917612498)
category_id_Entertainment: (0.017341982753274688)
category_id_How to and Style: (0.014319522021548904)
trending_year: (0.013020485625248112)
category_id_People and Blogs: (0.012895376847492958)
category_id_Education: (0.012824742825579301)
category_id_News and Politics: (0.012281606153924669)
category_id_Science and Technology: (0.012210433407505496)
category_id_Film and Animation: (0.011736982944121318)
published_year: (0.011331978589265932)
category_id_Comedy: (0.010044891796278795)
category_id_Sports: (0.009219078000730466)
category_id_Pets and Animals: (0.006235990491829271)
category_id_Gaming: (0.004870938729689641)
category_id_Travel and Events: (0.0039537533702102565)
comments_disabled_"False": (0.003618090701294618)
category_id_Autos and Vehicles: (0.00347775904905821)
comments_disabled_"Tr